In [1]:
import torch

import mowl
mowl.init_jvm('10g')
from mowl.datasets import Dataset
from org.semanticweb.owlapi.apibinding import OWLManager
import java.io

from src.utils import *
from src.elmodule import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [3]:
# dataset_name = 'family'
# dataset_name = 'pizza'
dataset_name = 'OWL2DL-1'
# dataset_name = 'lubm_filtered'

In [4]:
experiments = get_experimets(dataset_name)

**Models**

**Eval**

**Experiments: Box2EL**

In [5]:
for experiment in experiments: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']                                                                                                               

    train_manager = OWLManager.createOWLOntologyManager()
    test_manager = OWLManager.createOWLOntologyManager()
    val_manager = OWLManager.createOWLOntologyManager()

    train_ontology = train_manager.loadOntologyFromOntologyDocument(java.io.File(f'datasets/{file_name}_train.owl')) # we add noise to train
    test_ontology = test_manager.loadOntologyFromOntologyDocument(java.io.File(f'datasets/{dataset_name}_test.owl'))
    val_ontology = val_manager.loadOntologyFromOntologyDocument(java.io.File(f'datasets/{dataset_name}_val.owl'))

    train_ont = preprocess_ontology_el(train_ontology)
    test_ont = preprocess_ontology_el(test_ontology)
    valid_ont = preprocess_ontology_el(val_ontology)

    dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)
    
    box2el = ElModel(dataset, 
                     module_name='box2el', 
                     dim=200, 
                     margin=0.1, 
                     batch_size=4096*8, 
                     test_batch_size=32, 
                     epochs=500, 
                     learning_rate=0.001,
                     device=device)
    
    box2el._train()
    
    print(f'{file_name}:')
    print('Membership:')
    metrics_membership = box2el._eval('membership')
    print('Subsumption:')
    metrics_subsumption = box2el._eval('subsumption')
    print('Link Prediction:')
    metrics_link_prediction = box2el._eval('link_prediction')
    print()
    save_results(metrics_subsumption, metrics_membership, metrics_link_prediction, f'models/results/box2el/{file_name}.txt')

    break

INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(161 94)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(162 95)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(137 96)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(172 128)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyRa' object has no attribute 'getSubClass'
INFO:root:Reverse translation. Ignoring axiom: FunctionalObjectProperty*(53)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '53'.
INFO:root:Reverse translation. Ignoring axiom: RangeAxiom*(163 96)
INFO:root:'uk.ac.manchester.cs.owl.owlapi.O

Epoch: 0, Training: EL loss: 3.7235, ABox loss (Membership): 0.8371, ABox loss (Link Prediction): 5.6942
Epoch: 25, Training: EL loss: 3.5798, ABox loss (Membership): 0.7879, ABox loss (Link Prediction): 5.6831
Epoch: 50, Training: EL loss: 3.4785, ABox loss (Membership): 0.7059, ABox loss (Link Prediction): 5.8657
Epoch: 75, Training: EL loss: 3.4083, ABox loss (Membership): 0.5493, ABox loss (Link Prediction): 6.3571
Epoch: 100, Training: EL loss: 3.3439, ABox loss (Membership): 0.3606, ABox loss (Link Prediction): 7.1751
Epoch: 125, Training: EL loss: 3.2815, ABox loss (Membership): 0.2237, ABox loss (Link Prediction): 8.1224
Epoch: 150, Training: EL loss: 3.2417, ABox loss (Membership): 0.1519, ABox loss (Link Prediction): 8.9536
Epoch: 175, Training: EL loss: 3.1759, ABox loss (Membership): 0.1174, ABox loss (Link Prediction): 9.5977
Epoch: 200, Training: EL loss: 3.1368, ABox loss (Membership): 0.0991, ABox loss (Link Prediction): 10.1000
Epoch: 225, Training: EL loss: 3.0814, AB

In [40]:
_, labels, property_labels = box2el.get_abox_data("test")
ds = torch.nonzero(property_labels).squeeze()  
individuals = ds[:, 0]  
individuals2 = ds[:, 1]  
eval_dl = FastTensorDataLoader(individuals, individuals2, batch_size=box2el.test_batch_size, shuffle=False)     

In [7]:
ranks = dict()
rank_vals = []

mrr = 0
hits_at_1 = 0
hits_at_5 = 0
hits_at_10 = 0

for head_idxs, tail_idxs in eval_dl:
    predictions = box2el.predict(head_idxs, tail_idxs, mode='link_prediction')
    for i, head in enumerate(head_idxs):
        tail = tail_idxs[i]
        preds = predictions[i]
        orderings = torch.argsort(preds, descending=True)
        rank = torch.where(orderings == tail)[0].item()
        rank_vals.append(rank)
        if rank not in ranks:
            ranks[rank] = 0
        ranks[rank] += 1

        mrr += 1/(rank+1)
        if rank < 1:
            hits_at_1 += 1
        if rank < 5:
            hits_at_5 += 1
        if rank < 10:
            hits_at_10 += 1
            
mrr /= eval_dl.dataset_len

In [42]:
labels.size()

torch.Size([3643, 133])

In [47]:
torch.nonzero(labels).squeeze().size()

torch.Size([3276, 2])

In [43]:
property_labels.size()

torch.Size([3643, 3643])

In [44]:
g_test = rdflib.Graph()
g_test.parse('datasets/OWL2DL-1_test.owl', format='turtle')

<Graph identifier=N5a71331ef46e4440ad1b6b67ad396eb4 (<class 'rdflib.graph.Graph'>)>

In [45]:
from rdflib import RDF
len(list(g_test.triples((None, RDF.type, None))))

4286

In [48]:
list(g_test.triples((None, RDF.type, None)))

[(rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Thing'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')),
 (rdflib.term.URIRef('https://kracr.iiitd.edu.in/OWL2Bench#CoEdCollege'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')),
 (rdflib.term.URIRef('https://kracr.iiitd.edu.in/OWL2Bench#Dean'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')),
 (rdflib.term.URIRef('https://kracr.iiitd.edu.in/OWL2Bench#Director'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')),
 (rdflib.term.URIRef('https://kracr.iiitd.edu.in/OWL2Bench#Institute'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/20

In [8]:
mrr

0.044788577827771674

In [ ]:
eval_dl.dataset_len # not correct ? expected around 3-4k

17530

In [13]:
ds.size()

torch.Size([17530, 2])

In [14]:
ds

tensor([[   2,    2],
        [   3,    3],
        [   4,    4],
        ...,
        [3637, 3598],
        [3637, 3599],
        [3637, 3637]])

In [15]:
ontology = box2el.dataset.testing

In [16]:
property_assertions = []
object_properties = list(ontology.getObjectPropertiesInSignature())

In [18]:
len(object_properties)

79

In [19]:
individuals = ontology.getIndividualsInSignature()

In [20]:
len(individuals)

3416

In [21]:
for individual in individuals:
    for prop in object_properties:
        assertions = ontology.getObjectPropertyAssertionAxioms(individual)
        for assertion in assertions:
            if assertion.getProperty().equals(prop):  
                property_assertions.append(assertion)

In [22]:
len(property_assertions)

27778

In [23]:
property_assertions

[<java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAssertionAxiomImpl'>,
 <java object 'uk.ac.manchester.cs.owl.owlapi.OWLObjectPropertyAs

In [25]:
abox = []
for cls in dataset.classes:
    abox.extend(list(ontology.getClassAssertionAxioms(cls)))  

In [27]:
len(abox)

3276

rdflib.term.URIRef('http://www.w3.org/2002/07/owl#NamedIndividual')

In [35]:
from rdflib import OWL

property_assertions = []
object_properties = list(ontology.getObjectPropertiesInSignature())
individuals = ontology.getIndividualsInSignature()

for individual in individuals:
    for prop in object_properties:
        assertions = ontology.getObjectPropertyAssertionAxioms(individual)
        for assertion in assertions:
            if assertion.getProperty().equals(prop):  
                # Check if the object is an OWLNamedIndividual in mowl
                if isinstance(assertion.getObject(), OWL.NamedIndividual):  # Correct class check
                    property_assertions.append(assertion)


TypeError: isinstance() arg 2 must be a type or tuple of types

In [36]:
assertion.getObject()

<java object 'uk.ac.manchester.cs.owl.owlapi.OWLNamedIndividualImpl'>

In [37]:
object = assertion.getObject()
print(f"Object: {object}")

Object: <https://kracr.iiitd.edu.in/OWL2Bench#Architecture>


In [38]:
object

<java object 'uk.ac.manchester.cs.owl.owlapi.OWLNamedIndividualImpl'>